In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import spacy
import os
import regex as re
import json
from pymarc import MARCReader
import requests
from difflib import SequenceMatcher
import Levenshtein
from SPARQLWrapper import SPARQLWrapper, JSON



nlp = spacy.load("pl_core_news_lg")

print("Done")

Done


In [2]:
def query_wikidata(term):
    try:
        sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        query = """
            SELECT distinct ?item ?itemLabel ?label_en ?label_fr ?label_ger 
            ?label_ukr ?itemDescription ?lcsh ?unesco ?eurovoc ?YSO ?openalex 

            WHERE{  
              ?item ?label "%s"@pl .  
              OPTIONAL {?item wdt:P244 ?lcsh . }
              OPTIONAL {?item wdt:P757 ?unesco . }
              OPTIONAL {?item wdt:P5437 ?eurovoc . }
              OPTIONAL {?item wdt:P10283 ?openalex . }
              OPTIONAL {?item wdt:P2347 ?YSO . }
              ?item rdfs:label ?label_en filter (lang(?label_en) = "en").
              ?item rdfs:label ?label_fr filter (lang(?label_fr) = "fr").
              ?item rdfs:label ?label_ger filter (lang(?label_ger) = "de").
              ?item rdfs:label ?label_ukr filter (lang(?label_ukr) = "uk").
              SERVICE wikibase:label { bd:serviceParam wikibase:language "pl". }    
            }
        """ % (term)
        sparql.setQuery(query)

        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        if results["results"]["bindings"]:
            results_df = pd.json_normalize(results['results']['bindings'])
            return results_df
    except:
        pass

In [3]:
def modify_df(df):
    dict_of_single_col = {}
    for column in columns_of_interest:
        if column in df.columns:
            dict_of_single_col[column] = str(df[column].to_list())
            final_df = pd.DataFrame(x_df, index = [0])
            
    return final_df

In [4]:
litznawstwo = pd.read_csv("Kategorie tematyczne/df_literaturoznawstwo.csv").rename(columns = {"Unnamed: 0": "150"})

In [5]:
tqdm.pandas()
litznawstwo["wikidata"] = litznawstwo["150"].progress_apply(query_wikidata)

100%|█████████████████████████████████████████| 824/824 [03:06<00:00,  4.41it/s]


In [11]:
litznawstwo["wikidata"][2]

,item.type,item.value,label_en.xml:lang,label_en.type,label_en.value,label_fr.xml:lang,label_fr.type,label_fr.value,label_ger.xml:lang,label_ger.type,label_ger.value,label_ukr.xml:lang,label_ukr.type,label_ukr.value,itemLabel.xml:lang,itemLabel.type,itemLabel.value,lcsh.type,lcsh.value
0,uri,http://www.wikidata.org/entity/Q160476,en,literal,German Expressionism,fr,literal,cinéma expressionniste,de,literal,Expressionismus,uk,literal,Експресіонізм у кіно,pl,literal,Ekspresjonizm,NaN,NaN
1,uri,http://www.wikidata.org/entity/Q1362030,en,literal,expressionism,fr,literal,Expressionnisme,de,literal,Expressionismus,uk,literal,Експресіонізм (література),pl,literal,Ekspresjonizm,literal,sh94004146


In [ ]:
lit_df = df.copy()
#lit_df = pd.read_csv("lit_with_wiki.csv")
lit_terms = list(lit_df["150"])

lit_df1 = lit_df.reset_index()
final_df1 = final_df.reset_index()

In [ ]:
new_df = pd.concat([lit_df1, final_df1], axis = 1)